## Import needed library

In [1]:
from pyspark import SQLContext, SparkContext
from pyspark.sql.functions import mean, col, udf
from pyspark.sql.types import *
import numpy as np
import pandas
import findspark
findspark.init()

spark_context = SparkContext('local')
sqlc = SQLContext(spark_context)
train = sqlc.read.csv('data/train.csv', sep='\t', header=True)
test = sqlc.read.csv('data/test.csv', sep='\t', header=True)

22/12/14 22:39:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Split Problem Hieararchy

In [2]:
@udf
def get_unit(str):
    return str.split(",")[0]

@udf
def get_section(str):
     return str.split(",")[1]

train = train.withColumn("Problem Unit",get_unit(train["Problem Hierarchy"]))
train = train.withColumn("Problem Section",get_section(train["Problem Hierarchy"]))
test = test.withColumn("Problem Unit",get_unit(test["Problem Hierarchy"]))
test = test.withColumn("Problem Section",get_section(test["Problem Hierarchy"]))
# train = train.drop('Problem Hierarchy')
# test = test.drop('Problem Hierarchy')

## get the number of KC

In [3]:
@udf
def KC_num(str):
    if str:
        return str.count("~~")+1
    else:
        return 0
train = train.withColumn("KC num",KC_num(train['KC(Default)']))
test = test.withColumn("KC num",KC_num(test['KC(Default)']))

## get oportunity min and avg

In [4]:
@udf 
def opp_avg(str):
    if str:
        str_list = str.split("~~")
        num_list = []
        for s in str_list:
            num = int(s)
            num_list.append(num)
        return 1.0 * sum(num_list)/len(num_list)
    else:
        return 0
train = train.withColumn("Opportunity Average",opp_avg(train['Opportunity(Default)']))
test = test.withColumn("Opportunity Average",opp_avg(test['Opportunity(Default)']))

@udf 
def opp_min(str):
    if str:
        str_list = str.split("~~")
        num_list = []
        for s in str_list:
            num = int(s)
            num_list.append(num)
        return min(num_list)
    else:
        return 0
train = train.withColumn("Opportunity Min",opp_min(train["Opportunity(Default)"]))
test = test.withColumn("Opportunity Min",opp_min(test["Opportunity(Default)"]))


## Encode 

In [6]:
def encode(column):
    global train,test
    index = {}
    all_rows = train.union(test).select(column).distinct().collect()
    col_value = []
    for item in all_rows:
        col_value.append(item[column])
    for i, val in enumerate(col_value):
        index[val] = i
    @udf
    def get_id(str):
        return index[str]
    
    encoded_column = "encoded" + " " + column 
    train = train.withColumn(encoded_column, get_id(train[column]))
    train = train.withColumnRenamed(encoded_column, column)
    test = test.withColumn(encoded_column, get_id(test[column]))
    test = test.withColumnRenamed(encoded_column, column)

col_encode = ['Anon Student Id','Problem Name','Problem Unit','Problem Section','Step Name']
for col in col_encode:
    encode(col)


In [7]:
t = train.toPandas()
t.head()

22/12/14 22:41:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,Problem Unit,Problem Section,KC num,Opportunity Average,Opportunity Min,Anon Student Id,Problem Name,Problem Unit,Problem Section,Step Name
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,...,Unit CTA1_13,Section CTA1_13-1,0,0,0,170,69,15,33,34511
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,...,Unit CTA1_13,Section CTA1_13-1,0,0,0,170,69,15,33,14387
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,...,Unit CTA1_13,Section CTA1_13-1,1,1.0,1,170,69,15,33,58515
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,...,Unit CTA1_13,Section CTA1_13-1,3,1.0,1,170,69,15,33,60641
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,...,Unit CTA1_13,Section CTA1_13-1,1,1.0,1,170,69,15,33,30202
